In [ ]:
using JuMP, Gurobi, CSV, DataFrames

# -------------------------------
# Load Data and Setup Parameters
# -------------------------------

file_path = "Data/Final_Formatted_Sample_Day.csv"
df = CSV.read(file_path, DataFrame)

walking_distances_file = "Data/Walking Distances Arriving and Departing Pax.csv"
walking_distances = CSV.read(walking_distances_file, DataFrame)

# Separate arriving and departing flights
departing_indices = findall(df.IsDeparting .== "Y")
arriving_indices  = findall(df.IsDeparting .== "N")
F_dep = length(departing_indices)  # Number of departing flights
F_arr = length(arriving_indices)     # Number of arriving flights
F     = nrow(df)                     # Total flights
G     = 96                         # Number of gates

# Define enter and exit gate times
df[!, :EnterGateTime] = df.ArrivalTimeMinutes
df[!, :ExitGateTime]  = df.OffTimeMinutes

BUFFER_TIME = 0   # Buffer time (modifiable parameter)

# Walking times to gates from TSA
W_g = walking_distances.TSA_to_Gate

# Passenger count for flight f (if PassengersArr > 0 then use it, else use PassengersDept)
P_f = [ df.PassengersArr[f] > 0 ? df.PassengersArr[f] : df.PassengersDept[f] for f in 1:F ]

# -------------------------------
# Define the Model
# -------------------------------

model = Model(Gurobi.Optimizer)
@variable(model, M[1:F, 1:G], Bin)

# Objective: Minimize total walking time for departing flights
# (Note: the departing flights are referenced via departing_indices)
@objective(model, Min, 
    sum(W_g[g] * P_f[f] * M[departing_indices[f], g] for f in 1:F_dep, g in 1:G)
)

# Each flight is assigned exactly one gate
@constraint(model, [f in 1:F], sum(M[f, g] for g in 1:G) == 1)

# -------------------------------
# Precompute Conflict Pairs
# -------------------------------

# These are pairs of flights (f1,f2) that overlap in time 
# (with a buffer added to the exit time) and belong to different aircraft.
conflict_pairs = Vector{Tuple{Int, Int}}()
for f1 in 1:(F-1)
    for f2 in (f1+1):F
        if df.TailNumber[f1] != df.TailNumber[f2]
            enter1  = df.EnterGateTime[f1]
            depart1 = df.ExitGateTime[f1] + BUFFER_TIME
            enter2  = df.EnterGateTime[f2]
            depart2 = df.ExitGateTime[f2] + BUFFER_TIME
            if (enter1 < depart2) && (enter2 < depart1)
                push!(conflict_pairs, (f1, f2))
            end
        end
    end
end

# Add constraints: no two conflicting flights may be assigned to the same gate.
for (f1, f2) in conflict_pairs
    for g in 1:G
        @constraint(model, M[f1, g] + M[f2, g] <= 1)
    end
end

# -------------------------------
# Precompute Same‐Gate Pairs for Connections
# -------------------------------

# These are pairs where an arriving flight and a departing flight
# (with the same tail number) must be assigned the same gate 
# if the departing flight’s start time is within 2 hours of the arriving flight’s exit.
same_gate_pairs = Vector{Tuple{Int, Int}}()
for f1 in arriving_indices
    for f2 in departing_indices
        if df.TailNumber[f1] == df.TailNumber[f2] && (df.ExitGateTime[f1] + 120 >= df.EnterGateTime[f2])
            push!(same_gate_pairs, (f1, f2))
        end
    end
end

# Add same‐gate constraints
for (f1, f2) in same_gate_pairs
    for g in 1:G
        @constraint(model, M[f1, g] == M[f2, g])
    end
end

# -------------------------------
# Solve the Model
# -------------------------------

optimize!(model)


# Departing flights only

In [43]:
using JuMP, Gurobi, CSV, DataFrames

# Load data
file_path = "Data/Final_Formatted_Sample_Day.csv"
df = CSV.read(file_path, DataFrame)

walking_distances_file = "Data/Walking Distances Arriving and Departing Pax.csv"
walking_distances = CSV.read(walking_distances_file, DataFrame)

# Filter df to be only rows with isDeparting = "Y"
df = filter(row -> row.IsDeparting == "Y", df)
F = size(df, 1)
G = 96

# Define enter and exit gate times
df[!, :EnterGateTime] = df.ArrivalTimeMinutes
df[!, :ExitGateTime] = df.OffTimeMinutes

# Buffer time (modifiable parameter)
BUFFER_TIME = 0  # Example: 0 min buffer

# Walking times to gates from TSA
W_g = walking_distances.TSA_to_Gate

P_f = [df.PassengersArr[f] > 0 ? df.PassengersArr[f] : df.PassengersDept[f] for f in 1:F]

# Define the optimization model
model = Model(Gurobi.Optimizer)

# Decision variables: Binary matrix M[f, g]
@variable(model, M[1:F, 1:G], Bin)

# Objective: Minimize total walking time for passengers on departing flights
@objective(model, Min, sum(W_g[g] * P_f[f] * M[f, g] for f in 1:F, g in 1:G))

# Constraint: Each flight must be assigned exactly one gate
@constraint(model, [f in 1:F], sum(M[f, g] for g in 1:G) == 1)

# Constraint: No two flights can occupy the same gate at the same time
for g in 1:G
    for f1 in 1:F-1
        enter1 = df.EnterGateTime[f1]
        depart1 = df.ExitGateTime[f1] + BUFFER_TIME
        
        for f2 in f1+1:F
            enter2 = df.EnterGateTime[f2]
            depart2 = df.ExitGateTime[f2] + BUFFER_TIME
            
            if (enter1 < depart2) && (enter2 < depart1)  # Overlapping times
                @constraint(model, M[f1, g] + M[f2, g] <= 1)
            end
        end
    end
end

# Solve the model
optimize!(model)


Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-27
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.6.0 23G93)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2276792 rows, 60672 columns and 4612992 nonzeros
Model fingerprint: 0x23e60be2
Variable types: 0 continuous, 60672 integer (60672 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+03, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 8839950.0000
Presolve removed 2206464 rows and 0 columns
Presolve time: 1.27s
Presolved: 70328 rows, 60672 columns, 597408 nonzeros
Variable types: 0 continuous, 60672 integer (60672 binary)
Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 0.86s

Barrier statistics:
 AA' NZ

In [56]:
# Extract results
assignments = Dict(f => g for f in 1:F, g in 1:G if value(M[f, g]) ≈ 1)

# Create new columns for optimized gate assignments
df[!, :OptDepGate] = Vector{Union{String, Missing}}(missing, nrow(df))
df[!, :OptArrGate] = Vector{Union{String, Missing}}(missing, nrow(df))

# Gate mapping dictionary
gate_mapping = Dict(
    1 => "A8", 2 => "A9", 3 => "A10", 4 => "A11", 5 => "A13", 
    6 => "A14", 7 => "A15", 8 => "A16", 9 => "A17", 10 => "A18", 
    11 => "A19", 12 => "A20", 13 => "A21", 14 => "A22", 15 => "A23", 
    16 => "A24", 17 => "A25", 18 => "A28", 19 => "A29", 20 => "A33", 
    21 => "A34", 22 => "A35", 23 => "A36", 24 => "A37", 25 => "A38", 
    26 => "A39", 27 => "B1", 28 => "B2", 29 => "B3", 30 => "B4", 
    31 => "B5", 32 => "B6", 33 => "B7", 34 => "B9", 35 => "B10", 
    36 => "B11", 37 => "B12", 38 => "B14", 39 => "B16", 40 => "B17", 
    41 => "B18", 42 => "B19", 43 => "B21", 44 => "B22", 45 => "B24", 
    46 => "B25", 47 => "B26", 48 => "B27", 49 => "B28", 50 => "B29", 
    51 => "B30", 52 => "B31", 53 => "B32", 54 => "B33", 55 => "B34", 
    56 => "B35", 57 => "B36", 58 => "B37", 59 => "B38", 60 => "B39", 
    61 => "B40", 62 => "B42", 63 => "B43", 64 => "B44", 65 => "B46", 
    66 => "B47", 67 => "B48", 68 => "B49", 69 => "C2", 70 => "C4", 
    71 => "C6", 72 => "C7", 73 => "C8", 74 => "C10", 75 => "C11", 
    76 => "C12", 77 => "C14", 78 => "C15", 79 => "C16", 80 => "C17", 
    81 => "C19", 82 => "C20", 83 => "C21", 84 => "C22", 85 => "C24", 
    86 => "C26", 87 => "C27", 88 => "C28", 89 => "C29", 90 => "C30", 
    91 => "C31", 92 => "C33", 93 => "C35", 94 => "C36", 95 => "C37", 
    96 => "C39"
)

# Assign gates
for f in 1:F
    gate_number = get(assignments, f, missing)
    if !ismissing(gate_number)
        gate_code = get(gate_mapping, gate_number, missing)
        if df.IsDeparting[f] == "Y"
            df[f, :OptDepGate] = gate_code
        else
            df[f, :OptArrGate] = gate_code
        end
    end
end

println(df)

# Save results
CSV.write("Optimized_Gate_Assignments_Sample_Day.csv", df)

100×31 DataFrame
 Row │ FlightNumber  IsDeparting  Destination  Origin   ArrivalTime  OffTime  ArrivalTimeMinutes  OffTimeMinutes  ArrivalTimeActual  OffTimeActual  ArrivalTimeActualMinutes  OffTimeActualMinutes  Aircraft   FlightType  PassengersArr  PassengersDept  DepGate   ArrGate   SchedDepLocal        SchedArrLocal        OutGateLocal         InGateLocal          SchedDepUtc       SchedArrUtc       OutGateUtc        InGateUtc         TailNumber  EnterGateTime  ExitGateTime  OptDepGate  OptArrGate
     │ String7       String1      String3      String3  Int64        Int64    Int64               Int64           Int64              Int64          Float64                   Float64               Float64?   String15?   Int64          Int64           String3?  String3?  String31             String31             String31             String31             String31          String31          String31          String31          String7     Int64          Int64         String?     String?
─────┼

"Optimized_Gate_Assignments_Sample_Day.csv"